<a href="https://colab.research.google.com/github/iptkachev/yda_crowdsourcing/blob/main/object_detection_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Annotating ground truth for object detection

The goal of this notebook is to annotate images which can later be used for training of detection algorithms.

We will configure and run such project in Toloka from scratch.

Performers will be asked to annotate the particular objects in the image.


To get acquainted with Toloka tools for free, you can use the promo code **TOLOKAKIT1** on $20 on your [profile page](https://toloka.yandex.com/requester/profile?utm_source=github&utm_medium=site&utm_campaign=tolokakit) after registration.

## The challenge
We have a set of real-life photos of roads:

<table  align="center">
  <tr><td>
    <img src="https://tlk.s3.yandex.net/sdc/photos/0b35956a9afc639a71045f09745096de.jpg"
         alt="Sample road photo"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> Sample road photo
  </td></tr>
</table>

We need to outline every traffic sign. Ultimately, we need to get a set of contours, defined by an array of points, that
represent the road signs in each photo. Here’s how it can look like:

<table  align="center">
  <tr><td>
    <img src="https://github.com/Toloka/toloka-kit/blob/main/examples/1.computer_vision/object_detection/img/segmentation_example.png?raw=1"
         alt="Example of how road sign detection can be performed"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 2.</b> Example of how road sign detection can be performed.
  </td></tr>
</table>

In real-world tasks, annotation is usually done with a polygon. We chose to use a rectangular outline to simplify the
task so that we can reduce costs and speed things up.

### Detailed task description
In this notebook we will implement Projects 2 and 3 from the [tutorial](https://toloka.ai/docs/guide/concepts/image-segmentation-overview.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit).
Check this link if you want to configure these projects right in the web interface.

We'll skip the first project "Does the image contain a specific object?" from tutorial above, since it's easy to
implement using our ["verification project"](https://toloka.ai/docs/guide/concepts/image-segmentation-project3.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) code.

Here are the two projects we’re going to implement:
- **Detection project** "[Select an object in the image](https://toloka.ai/docs/guide/concepts/image-segmentation-project2.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit)" : performers will select image areas that contain a traffic sign.
- **Verification project** "[Are the bounding boxes correct?](https://toloka.ai/docs/guide/concepts/image-segmentation-project3.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit)" : performers will determine if traffic signs were selected correctly in the images.

[Control tasks](https://toloka.ai/docs/guide/concepts/goldenset.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) and [majority vote](https://toloka.ai/docs/guide/concepts/mvote.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) aren't used for the detection project, because we can’t expect the area annotations provided by the performers to match each other exactly. Instead, we’ll check detection results in the second project, where a different group of performers will determine whether the traffic signs were annotated correctly or not.

### Set up the environment
First of all, we higly recommend to read [learn the basics example](https://github.com/Toloka/toloka-kit/blob/main/examples/0.getting_started/0.learn_the_basics/) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/0.getting_started/0.learn_the_basics/learn_the_basics.ipynb) before you start this one to be familiar with the main kinds of entities in Toloka.

In our example we are using the production version of Toloka, but you can also use [Toloka sandbox](https://toloka.ai/docs/guide/concepts/sandbox.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit). Sandbox specifications of the code are presented in the comments.

Prepare environment and import necessary libraries

In [1]:
!pip install toloka-kit==0.1.17 # To interact with Toloka API
!pip install ipyplot # To plot images inside Jupyter Notebooks cells
!pip install crowd-kit==0.0.7

import os
import datetime
import time
import logging
import sys

import pandas as pd   # To perform data manipulation
import ipyplot


from typing import List
from toloka.streaming.event import AssignmentEvent

import toloka.client as toloka
import toloka.client.project.template_builder as tb

from crowdkit.aggregation import MajorityVote

logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 299 kB 5.2 MB/s 
     |████████████████████████████████| 130 kB 45.1 MB/s 
     |████████████████████████████████| 1.6 MB 52.9 MB/s 
     |████████████████████████████████| 9.8 MB 40.5 MB/s 
     |████████████████████████████████| 357 kB 51.8 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=5a5ceaee537a5511f489a840433947e3ab72fcae726f330da79690ffb96afa6f
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 2.8 MB/s 

        WARNING! Google Colab Environment detected!
        You migh

Create a toloka-client instance. All API calls will pass through it.

In [2]:
toloka_client = toloka.TolokaClient(input("Enter your token:"), 'PRODUCTION') # Or switch to 'SANDBOX'
print(toloka_client.get_requester())

Enter your token:y0_AgAAAABk4TOnAACtpQAAAADQ-Q1gt4SSHYhrRS2IjTuE4SP_wzuFmd0
Requester(_unexpected={}, id='096191d20520f94d0fffa98499658f97', balance=Decimal('48.9548'), public_name={'EN': 'skaatt67'}, company=None)


Learn more about [Toloka API](https://toloka.ai/docs/api/concepts/about.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) and [Toloka Kit](https://toloka.github.io/toloka-kit?utm_source=github&utm_medium=site&utm_campaign=tolokakit).


### Review the dataset
The dataset used is collected by Toloka team and distributed under a Creative Commons Attribution 4.0 International license
[![License: CC BY 4.0](https://img.shields.io/badge/License-CC%20BY%204.0-lightgrey.svg)](https://creativecommons.org/licenses/by/4.0/).

Our dataset is just a collection of image URLs.

In [3]:
!curl https://tlk.s3.yandex.net/dataset/road_signs.tsv --output dataset.tsv

# Load the dataset of links to a pd DataFrame
dataset = pd.read_csv('dataset.tsv', sep='\t')

# Plot 5 images from dataset to verify data loading
ipyplot.plot_images(
    [url for url in dataset['image'].sample(n=50)],
    max_images=5,
    img_width=1000
)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6296  100  6296    0     0   8531      0 --:--:-- --:--:-- --:--:--  8531


---
---
## Create a new detection project

In this project, performers select image areas that contain traffic signs.

The first step is to configure how performers will see the tasks:
* write instructions,
* define the input and output formats.

**Note:** It's important to write clear instructions with examples to make sure the performers do exactly what we want. We also recommend checking the task interface.

In [4]:
# How performers will see the task
project_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ImageAnnotationFieldV1(  # Component that selects areas in images
        tb.OutputData('result'),  # Path for writing output data
        tb.InputData('image'),  # Getter for the input image
        shapes={'rectangle': True},  # Allow to select only rectangular areas
        validation=tb.RequiredConditionV1(hint='Please select an area')  # At least one area should be selected
    )
)

# You can write instructions and upload them from a file or enter them later in the web interface
# prepared_instruction = open('instruction.html').read().strip()
prepared_instruction = '<b>Outline each traffic sign with a separate bounding box(rectangle).</b>'

# Set up the project
detection_project = toloka.Project(
    public_name='Outline all traffic signs with bounding boxes',
    public_description='Find and outline all traffic signs with bounding boxes.',
    public_instructions=prepared_instruction,
    # Set up the task: view, input, and output parameters
    task_spec=toloka.project.task_spec.TaskSpec(
        input_spec={'image': toloka.project.UrlSpec()},
        output_spec={'result': toloka.project.JsonSpec()},
        view_spec=project_interface,
    ),
)

Call the API to create a new project.

In [4]:
try:
  detection_project = toloka_client.get_project(114335)
  print("Get project")
except toloka_client.exceptions.DoesNotExistApiError:
  detection_project = toloka_client.create_project(detection_project)

Get project


### Review your project and check the task interface

Visit the project page to make sure the task interface is working correctly.

To do this:

1. Follow the link in the output above.
2. In the project interface, click **Project actions** on the top right.
3. Click **Preview** in the menu that appears.
4. Click **Change input data**.
5. Insert an image URL (for example, `https://tlk.s3.yandex.net/sdc/photos/0b35956a9afc639a71045f09745096de.jpg`) into the `image` field.
6. Click the **Instructions** button. Make sure the instructions are shown and valid.
7. Try to select multiple areas with a rectangle using **Box annotation tool**.
8. Click **Submit** and then **View responses**.

The result window will appear. Check that your results are in the expected format and that the data is being entered correctly.

<table  align="center">
  <tr><td>
    <img src="https://github.com/Toloka/toloka-kit/blob/main/examples/1.computer_vision/object_detection/img/segmentation_results_preview.png?raw=1"
         alt="Task interface"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 3.</b> What the results window might look like
  </td></tr>
</table>

We strongly recommend to check the task interface and instructions every time you create a project. This helps performers to complete the task correctly and your results to be useful.

**Tip:** Do a trial run with a small amount of data. Make sure that after running the entire pipeline, you get data in the expected format and quality.

### Add custom skills for performers

A skill can describe any characteristic of the performer. Skills are defined by a number from 0 to 100. For example, you can record the percentage of correct responses as a skill. Learn more about [skills](https://toloka.ai/docs/guide/concepts/nav.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

In this project, we'll create two skills:
- **Detection skill**: Shows the performer completed at least one detection task. We'll later filter out these performers from verification tasks, so that no one can check their own detection.
- **Verification skill**: How good the current performer is, when compared to others. We'll need this skill later when aggregating the results of the second project.

In [7]:
detection_skill = next(toloka_client.get_skills(name='Area selection of road signs'), None)
if detection_skill:
    print('Detection skill already exists')
else:
    detection_skill = toloka_client.create_skill(
        name='Area selection of road signs',
        hidden=True,
        public_requester_description={'EN': 'Performer is annotating road signs'},
    )

verification_skill = next(toloka_client.get_skills(name='Verification skill'), None)
if verification_skill:
    print('Verification skill already exists')
else:
  verification_skill = toloka_client.create_skill(
      name='Verification skill',
      hidden=True,
      public_requester_description={'EN': 'Performer is annotating road signs'},
  )

Detection skill already exists
Verification skill already exists


### Pool creation for a detection project
A pool is a set of paid tasks sent out for performers.

First, create an instance of the pool and set the basic parameters:
- Payment amount per task.
- Non-automatic acceptance of results.
- Number of tasks performers will see on one page.
- Performer's filter: control who can access this task.

More about configuring and running pools in:
* [Toloka Kit](https://toloka.ai/docs/toloka-kit/source/toloka.client.pool.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit)
* [Help](https://toloka.ai/docs/guide/concepts/pool-main.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit)
* [Toloka API](https://toloka.ai/docs/api/create-pool.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit)

In [11]:
detection_pool = toloka.Pool(
    project_id=detection_project.id,
    private_name='Pool 1',  # Only you can see this information.
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),  # Pool will automatically close after one year
    reward_per_assignment=0.01,     # Set the minimum payment amount for one task page
    auto_accept_solutions=False,    # Only pay the performer for completing the task,
                                    # based on the verification results of the second project.

    auto_accept_period_day=7,       # Number of days to determine if we'll pay for task completion by this performer or not.
    assignment_max_duration_seconds=60*20,  # Give performers 20 minutes maximum to complete one task page.
    defaults=toloka.pool.Pool.Defaults(
        # We don't need overlapping for detection tasks, so we set it to 1
        default_overlap_for_new_task_suites=1,
        default_overlap_for_new_tasks=1,
    ),
)

# Set the number of tasks per page
detection_pool.set_mixer_config(real_tasks_count=1)
# Please note that the payment amount specified when creating the pool is the amount the performer receives for completing one page of tasks.
# If you specify 10 tasks per page above, then reward_per_assignment will be paid for completing 10 tasks.

We'll only show our tasks to English-speaking users because the description of the task is in English.
This means that only people who speak English will be able to accept this task.

In [12]:
detection_pool.filter = toloka.filter.Languages.in_('EN')

**Quality control rules**

View a detailed description of our quality control rules [in documentation](https://toloka.ai/docs/guide/concepts/control.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

Each quality control rule consists of the following:
- **Collector**: How to collect statistics and which metrics can be used in this rule.
- **Condition**: When the rule will be triggered. Under this condition, only parameters that apply to the collector can be used.
- **Action**: What to do if the condition is true.

In [13]:
# The first rule in this project restricts pool access for performers who often make mistakes
detection_pool.quality_control.add_action(
    collector=toloka.collectors.AcceptanceRate(),
    conditions=[
        # Performer completed more than 2 tasks
        toloka.conditions.TotalAssignmentsCount > 2,
        # And more than 35% of their responses were rejected
        toloka.conditions.RejectedAssignmentsRate > 35,
    ],
    # This action tells Toloka what to do if the condition above is True
    # In our case, we'll restrict access for 15 days
    # Always leave a comment: it may be useful later on
    action=toloka.actions.RestrictionV2(
        scope='ALL_PROJECTS',
        duration=15,
        duration_unit='DAYS',
        private_comment='Performer often make mistakes',  # Only you will see this comment
    )
)

# The second useful rule is "Fast responses". It allows us to filter out performers who respond too quickly.
detection_pool.quality_control.add_action(
    # Let's monitor fast submissions for the last 5 completed task pages
    # And define ones that take less than 20 seconds as quick responses.
    collector=toloka.collectors.AssignmentSubmitTime(history_size=5, fast_submit_threshold_seconds=20),
    # If we see more than one fast response, we ban the performer from all our projects for 10 days.
    conditions=[toloka.conditions.FastSubmittedCount > 1],
    action=toloka.actions.RestrictionV2(
        scope='ALL_PROJECTS',
        duration=10,
        duration_unit='DAYS',
        private_comment='Fast responses',  # Only you will see this comment
    )
)

# Another rule we use is for automatically updating skills
# We update the detection skill for performers who complete at least one page of tasks from detection pool.
detection_pool.quality_control.add_action(
    collector=toloka.collectors.AnswerCount(),
    # If performer completed at least one task, it sets the new skill to 1
    conditions=[toloka.conditions.AssignmentsAcceptedCount > 0],
    action=toloka.actions.SetSkill(skill_id=detection_skill.id, skill_value=1),
)

# This rule sends rejected assignments (tasks that you rejected) to other performers according to specified parameters.
detection_pool.quality_control.add_action(
    collector=toloka.collectors.AssignmentsAssessment(),
    # Check if a task was rejected
    conditions=[toloka.conditions.AssessmentEvent == 'REJECT'],
    # If the condition is True, add 1 to overlap and open the pool
    action=toloka.actions.ChangeOverlap(delta=1, open_pool=True),
)

print('Quality rules count:', len(detection_pool.quality_control.configs))

Quality rules count: 4


### Create a pool with all specified conditions

Now we call the Toloka API to finally create a pool in the detection project.

Afterwards, you can check the pool in the web interface. You'll see there aren't any tasks in it. We'll add them later.

In [14]:
detection_pool = toloka_client.create_pool(detection_pool)

---
---
## Create a new project for verification
In this project, performers will determine if traffic signs were outlined correctly or not.

This will be a standard classification project with only two classes: `OK` and `BAD`. We’ll explicitly define these labels as the output values.

In [16]:
# Configure task interface: how performers will see the task
verification_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ListViewV1(  # List of components that should be positioned from top to bottom in the UI
        [
            tb.ImageAnnotationFieldV1(  # Image and selected areas to verify
                tb.InternalData('selection',
                                default=tb.InputData('selection')),  # Use the input field as default value to display the selected areas
                tb.InputData('image'),
                disabled=True  # Disable adding and deleting areas
            ),
            tb.RadioGroupFieldV1(  # A component for selecting one value out of several options
                tb.OutputData('result'),  # Path for writing output data
                [
                    tb.GroupFieldOption('OK', 'Yes'),
                    tb.GroupFieldOption('BAD', 'No'),
                ],
                label='Are all traffic signs outlined correctly?',  # Label above the options
                validation=tb.RequiredConditionV1()  # Requirement to select one of the options
            )
        ]
    ),
    plugins=[
        tb.HotkeysPluginV1( # Shortcuts for selecting options using the keyboard
            key_1=tb.SetActionV1(tb.OutputData('result'), 'OK'),
            key_2=tb.SetActionV1(tb.OutputData('result'), 'BAD')
        )
    ]
)

# You can write instructions and upload them from a file or enter them later in the web interface
# prepared_instruction = open('instruction.html').read().strip()
verification_instruction = '''<b>Look at the image and answer the question:</b><br/>
Are all traffic signs outlined correctly?<br/>
If they are, click Yes.<br/>
If they aren't, click No.<br/>
For example, the road signs here are outlined correctly, so the correct answer is Yes.'''

# Set up the project
verification_project = toloka.Project(
    public_name='Are the traffic signs outlined correctly?',
    public_description='Look at the image and decide whether or not the traffic signs are outlined correctly',
    public_instructions=verification_instruction,
    # Set up the task: view, input, and output parameters
    task_spec=toloka.project.task_spec.TaskSpec(
        input_spec={
            'image': toloka.project.UrlSpec(),
            'selection': toloka.project.JsonSpec(),
            'assignment_id': toloka.project.StringSpec(),
        },
        # Set allowed_values, we'll use smart mixing to get the results of this project
        output_spec={'result': toloka.project.StringSpec(allowed_values=['OK', 'BAD'])},
        view_spec=verification_interface,
    ),
)

Call the API to create a new project

In [5]:
try:
  verification_project = toloka_client.get_project(114337)
  print("Get project")
except toloka_client.exceptions.DoesNotExistApiError:
  verification_project = toloka_client.create_project(verification_project)

Get project


Examine created project in the web interface. To do that:

1. Follow the link above to check the task interface and instructions.

    **Note:** You should see nearly the same interface as in the previous project, only without the ability to select areas.
    It's important to make sure that the annotation results from the first project display correctly in the second one.


2. Open the task **Preview** in the first project.
3. Outline the signs and click **Submit**.
4. Copy the result.
5. Now open the **Preview** of the second project.
6. Click **Change input data** and paste the annotation results in the `selection` field.
7. Click **Apply** and make sure the annotation displays correctly.

### Create and set up a pool in the verification project
We will add filter for this pool: specify performers that don't have the detection skill (as they performed in detection tasks). You can combine multiple conditions using the `&` and `|` operators.

**Note:** we add two quality control rules with the same collector, but with different conditions and actions.

In [9]:
verification_pool = toloka.Pool(
    project_id=verification_project.id,
    private_name='Pool 1.Road',  # Only you can see this information.
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),  # Pool will automatically close after one year
    reward_per_assignment=0.01,     # Set the minimum payment amount for one task page
    assignment_max_duration_seconds=60*20,  # Give performers 20 minutes maximum to complete one task page.
    defaults=toloka.pool.Pool.Defaults(
        # We don't need overlapping for detection tasks, so we set it to 1
        default_overlap_for_new_task_suites=5
    ),
) # <YOUR CODE>


# We'll only show our tasks to English-speaking users because the description of the task is in English.
# We also won't allow our verification tasks to be performed by users who performed detection tasks.

# language filter and lack of detection skill filter
# https://toloka.ai/en/docs/toloka-kit/reference/toloka.client.filter.Languages
# https://toloka.ai/en/docs/toloka-kit/reference/toloka.client.filter.Skill

verification_pool.filter = (
    toloka.filter.Languages.in_('EN') &
    (toloka.filter.Skill(detection_skill.id) == None)
) # <YOUR CODE>

# Set up quality control
# Quality is based on the majority of matching responses from performers who completed the same task.

# Считать большинством — 2. Смотреть 10 последних ответов. 
# Если количество ответов ≥ 3  и процент правильных ответов < 50, то заблокировать на проекте  на 7  дней
# https://toloka.ai/docs/toloka-kit/reference/toloka.client.collectors.MajorityVote.html?lang=en
# https://toloka.ai/en/docs/toloka-kit/reference/toloka.client.conditions.TotalAnswersCount
# https://toloka.ai/en/docs/toloka-kit/reference/toloka.client.conditions.CorrectAnswersRate
# https://toloka.ai/en/docs/toloka-kit/reference/toloka.client.actions.RestrictionV2

verification_pool.quality_control.add_action(
    collector=toloka.collectors.MajorityVote(history_size=10, answer_threshold=2),
    conditions=[toloka.conditions.TotalAnswersCount >= 3, toloka.conditions.CorrectAnswersRate < 50],
    action=toloka.actions.RestrictionV2(
        scope='ALL_PROJECTS',
        duration=7,
        duration_unit='DAYS',
        private_comment='Bad quality',  # Only you will see this comment
    )


) # <YOUR CODE>


# Set up the new skill value using MajorityVote.
# Depending on the percentage of correct responses, we increase the value of the performer's skill.
verification_pool.quality_control.add_action(
    collector=toloka.collectors.MajorityVote(answer_threshold=2, history_size=10),
    conditions=[
        toloka.conditions.TotalAnswersCount > 2,
    ],
    action=toloka.actions.SetSkillFromOutputField(
        skill_id=verification_skill.id,
        from_field='correct_answers_rate',
    ),
)
print(f'Quality rule count:{len(verification_pool.quality_control.configs)}')

Quality rule count:1


### Create a pool

In [10]:
# Set the task count for one page

# 10 заданий на странице, выдавать неполную страницу, если заданий не хватило
# https://toloka.ai/en/docs/toloka-kit/reference/toloka.client.pool.mixer_config.MixerConfig

verification_pool.set_mixer_config(
    real_tasks_count=10, force_last_assignment=True
) # <YOUR CODE>

verification_pool = toloka_client.create_pool(verification_pool)

---
---
## Add tasks to pools and run the projects
At this point, we have configured two projects, and now we can upload the real data that we want to annotate.

In [35]:
# https://toloka.ai/en/docs/toloka-kit/reference/toloka.client.task.Task.html
# Add 100 first images from the dataset

tasks = [
    toloka.Task(input_values={'image': url}, pool_id=detection_pool.id)
    for url in dataset['image'].head(100)
] # <YOUR CODE>

In [36]:
# Add tasks to a pool
toloka_client.create_tasks(tasks, allow_defaults=True)

detection_pool = toloka_client.open_pool(detection_pool.id)

Visit the pool page in the web interface and make sure everything is ok: the number of tasks is correct, the pool is running, and some tasks may already be completed.

<table  align="center">
  <tr><td>
    <img src="https://github.com/Toloka/toloka-kit/blob/main/examples/1.computer_vision/object_detection/img/segmentation_pool_look.png?raw=1"
         alt="Pool with tasks"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 4.</b> How a running pool may look.
  </td></tr>
</table>


Toloka performers work really fast, but they still need time to complete their tasks. We’ll use streaming to avoid having to wait until the detection pool closes completely.

Remember that you should also review the assignments of the detection pool in the web interface. More about [reviewing assignments](https://toloka.ai/docs/guide/concepts/offline-accept.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit).
You can view the status of the pool in the web interface, but this is not very convenient in a real-life project.

In [38]:
from toloka.streaming import AssignmentsObserver, Pipeline

In [39]:
# class for handling submissions in the detection pool
class DetectionSubmittedHandler:
    def __init__(self, client, verification_pool_id):
        self.client = client
        self.verification_pool_id = verification_pool_id

    # create new tasks for the verification pool
    def __call__(self, events: List[AssignmentEvent]) -> None:
        verification_tasks = [
            toloka.Task(
                pool_id=self.verification_pool_id,
                input_values={
                        'image': event.assignment.tasks[0].input_values['image'],
                        'selection': event.assignment.solutions[0].output_values['result'],
                        'assignment_id': event.assignment.id,
                }
            )
            for event in events
        ]
        self.client.create_tasks(verification_tasks, allow_defaults=True, open_pool=True)

In [40]:
# class for handling accepted tasks in the verification pool
class VerificationDoneHandler:
    def __init__(self, client, verification_skill_id):
        self.microtasks = pd.DataFrame([], columns=['task', 'label', 'performer'])
        self.client = client
        self.verification_skill_id = verification_skill_id

    # filter out tasks that already have enough overlap and aggregate the result
    def __call__(self, events: List[AssignmentEvent]) -> None:
        # Initializing data
        microtasks = pd.concat([self.microtasks, self.as_frame(events)])
        # get user skills for aggregation
        skills = pd.Series({
            skill.user_id: skill.value
            for skill in self.client.get_user_skills(skill_id=self.verification_skill_id)
        })

        # Filtering all microtasks that have overlap of 5
        microtasks['overlap'] = microtasks.groupby('task')['task'].transform('count')
        to_aggregate = microtasks[microtasks['overlap'] >= 5]
        microtasks = microtasks[microtasks['overlap'] < 5]
        aggregated = MajorityVote().fit_predict(to_aggregate, skills)
        # Accepting or rejecting assignments
        for assignment_id, result in aggregated.items():
            if result == 'OK':
                self.client.accept_assignment(assignment_id, 'Well done!')
            else:
                toloka_client.reject_assignment(assignment_id, 'The object wasn\'t selected or was selected incorrectly.')

        # Updating mictotasks
        self.microtasks = microtasks[['task', 'label', 'performer']]

    # get the data necessary for aggregation
    @staticmethod
    def as_frame(events: List[AssignmentEvent]) -> pd.DataFrame:
        microtasks = [
            (task.input_values['assignment_id'], solution.output_values['result'], event.assignment.user_id)
            for event in events
            for task, solution in zip(event.assignment.tasks, event.assignment.solutions)
        ]
        return pd.DataFrame(microtasks, columns=['task', 'label', 'performer'])

We'll create a pipeline with an observer for each pool.

Depending on the number of images in the detection pool and the time of day, the whole process can take from a few minutes to almost an hour.

In [ ]:
detection_observer = AssignmentsObserver(toloka_client, detection_pool.id)
detection_observer.on_submitted(DetectionSubmittedHandler(toloka_client, verification_pool.id))
verification_observer = AssignmentsObserver(toloka_client, verification_pool.id)
verification_observer.on_accepted(VerificationDoneHandler(toloka_client, verification_skill.id))

pipeline = Pipeline()
pipeline.register(detection_observer)
pipeline.register(verification_observer)

# Google Colab is using a global event pool,
# so in order to run our pipeline we have to apply nest_asyncio to create an inner pool
if 'google.colab' in str(get_ipython()):
    import nest_asyncio, asyncio
    nest_asyncio.apply()
    asyncio.get_event_loop().run_until_complete(pipeline.run())
else:
    await pipeline.run()

---
---
## Get the results
Now we can download all the accepted tasks from the detection pool and work with them. In this notebook, we'll only show the detection results.
You can also [download](https://toloka.ai/docs/guide/concepts/result-of-eval.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) results as a TSV file from web interface.

In [ ]:
!pip install pillow # To deal with images
!pip install requests # To make HTTP requests
from PIL import Image, ImageDraw
import requests

def get_image(url, selection):
    raw_image = requests.get(url, stream=True).raw
    image = Image.open(raw_image).convert("RGBA")
    regions = Image.new('RGBA', image.size, (255,255,255,0))
    pencil = ImageDraw.Draw(regions)
    for region in selection:
        if region['shape'] != 'rectangle':
            continue
        p1_x = region['left'] * image.size[0]
        p1_y = region['top'] * image.size[1]
        p2_x = (region['left'] + region['width']) * image.size[0]
        p2_y = (region['top'] + region['height']) * image.size[1]
        pencil.rectangle((p1_x, p1_y, p2_x, p2_y), fill =(255, 30, 30, int(255*0.5)))
    image = Image.alpha_composite(image, regions)
    return image

detection_result = {}  # We'll store our result here

In [ ]:
max_images = 2
images = []

if not detection_result:

    for assignment in toloka_client.get_assignments(
        status='ACCEPTED',
        pool_id=detection_pool.id
    ):
        detection_result[assignment.tasks[0].input_values['image']] = assignment.solutions[0].output_values['result']

for i in range(max_images):
    url, selection = detection_result.popitem()
    image = get_image(url, selection)
    images.append(image)

ipyplot.plot_images(
    images,
    max_images=max_images,
    img_width=1000
)

---
---
## Summary

This project consists of the minimum number of settings that will allow you to collect annotated images for your dataset right from Jupyter Notebook.

For your future experiments use Toloka Kit [documentation](https://toloka.ai/docs/toloka-kit/index.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) and check out other [use cases](https://github.com/Toloka/toloka-kit/tree/main/examples).
